In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_diagonal_wavelet_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,31047.76,-0.16,-0.02,0.10,0.00,-0.01,0.00,-0.0,-0.00
3,-0.16,6940.27,-0.04,0.02,-0.02,0.00,0.00,0.0,0.00
4,-0.02,-0.04,1469.35,0.02,-0.00,-0.00,-0.00,-0.0,0.00
5,0.10,0.02,0.02,290.61,0.00,-0.00,0.00,0.0,0.00
6,0.00,-0.02,-0.00,0.00,54.64,-0.00,-0.00,0.0,0.00
7,-0.01,0.00,-0.00,-0.00,-0.00,9.86,0.00,0.0,0.00
8,0.00,0.00,-0.00,0.00,-0.00,0.00,1.77,0.0,0.00
9,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.3,-0.00
10,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.0,0.04


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,-0.00000,0.00003,0.00000,-0.00002,0.00001,-0.00001,-0.00000
3,-0.00001,1.00000,-0.00001,0.00001,-0.00003,0.00001,0.00004,0.00000,0.00000
4,-0.00000,-0.00001,1.00000,0.00004,-0.00000,-0.00000,-0.00003,-0.00001,0.00001
5,0.00003,0.00001,0.00004,1.00000,0.00000,-0.00002,0.00005,0.00000,0.00000
6,0.00000,-0.00003,-0.00000,0.00000,1.00000,-0.00001,-0.00001,0.00002,0.00001
7,-0.00002,0.00001,-0.00000,-0.00002,-0.00001,1.00000,0.00002,0.00000,0.00000
8,0.00001,0.00004,-0.00003,0.00005,-0.00001,0.00002,1.00000,0.00003,0.00000
9,-0.00001,0.00000,-0.00001,0.00000,0.00002,0.00000,0.00003,1.00000,-0.00000
10,-0.00000,0.00000,0.00001,0.00000,0.00001,0.00000,0.00000,-0.00000,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.28278045223089826

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.07198955e+08 5.26926727e+06 2.31360666e+05 8.80041893e+03
 2.98742513e+02 9.12965269e+00 2.62735504e-01 5.83686615e-03
 2.24061493e-35]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999551,-0.029370,-0.005883,-0.001157,-0.000218,-0.000040,-0.000007,-0.000001,-1.631457e-07
1,0.029161,0.999050,-0.031808,-0.005977,-0.001124,-0.000202,-0.000036,-0.000006,-8.150312e-07
2,0.006760,0.031392,0.998871,-0.034437,-0.006193,-0.001116,-0.000202,-0.000035,-4.490899e-06
3,0.001552,0.006965,0.033948,0.998608,-0.039129,-0.006775,-0.001211,-0.000209,-2.739914e-05
4,0.000350,0.001570,0.007411,0.038519,0.998100,-0.046786,-0.008078,-0.001383,-1.820002e-04
5,0.000079,0.000352,0.001664,0.008425,0.045877,0.996932,-0.061983,-0.010193,-1.339539e-03
6,0.000019,0.000083,0.000398,0.002001,0.010672,0.060385,0.994153,-0.088155,-1.110031e-02
7,0.000005,0.000021,0.000101,0.000511,0.002720,0.015083,0.085143,0.987476,-1.319371e-01
8,0.000001,0.000006,0.000027,0.000136,0.000726,0.004022,0.022382,0.130441,9.911950e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.00044939581988756583,
 0.0009500292921269526,
 0.0011289994731459307,
 0.0013919391221663968,
 0.0019004518903060763,
 0.0030677928956949208,
 0.005846584708185665,
 0.012524414583695243,
 0.008804989669995433]